In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.models import NumeralTickFormatter
from bokeh.io import output_notebook
from datetime import timedelta
import numpy as np
from IPython.core.display import display, HTML
from bokeh.models import HoverTool
from bokeh.layouts import row,column,grid
from bokeh.plotting import figure, show

In [2]:
display(HTML("<h1 align='center'>Analysis of the Federal Debt</h1>"))

In [3]:
display(HTML("<h2 align='center'>Ryan Miller<br><br>05/26/2020</h2>"))

In [4]:
display(HTML("<h6>The loading symbol below starts the data visualization tool in the notebook.</h6>"))

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
paragraph_style=" style='text-align:justify;font-size:16px;'"

In [7]:
from fredapi import Fred

with open(r"C:/Users/ryana/Desktop/fred_api_key.txt") as f:
    api_key = f.read()
fred = Fred(api_key=api_key)

#Public Debt
federal_debt_to_gdp = fred.get_series('GFDEGDQ188S')
federal_debt_total = fred.get_series('GFDEBTN')
federal_debt_total = federal_debt_total / 1000

#Deficit
annual_federal_deficit = fred.get_series('FYFSGDA188S')
federal_deficit = fred.get_series('MTSDS133FMS')

#Debt Components
debt_by_foreigners = fred.get_series('FDHBFIN')
debt_by_fed_banks = fred.get_series("FDHBFRBN")
debt_by_private_investors = fred.get_series("FDHBPIN")
debt_by_public = fred.get_series("FYGFDPUN")
debt_by_agencies = fred.get_series("FDHBATN")
debt_by_agencies = debt_by_agencies / 1000
debt_by_public = debt_by_public / 1000

#GDP
nominal_gdp = fred.get_series('GDP')
government_consumption_expenditures = fred.get_series('GCE')

#Population
population = fred.get_series("POPTHM")
population_quarterly = population.groupby(pd.Grouper(freq="Q")).last()
population_quarterly.index = population_quarterly.index + timedelta(days=1)


In [8]:
intro_text = \
"""
The national debt level has been a significant subject of U.S. domestic policy controversy. 
As long as a government runs deficits, it has to find buyers for its bonds. 
It behooves every American to know how our tax money is being spent and the benefits/costs. 
Politicians, like President Trump, claim they will make efforts to reduce the debt. 
There have been few Presidents able to do so. 
The Legislative Branch of Congress is in charge of fiscal policy and the Executive Branch proposes yearly budget requests to fund agencies, bureaus, staffers, etc. 
Within this report, I will identify the relevant Federal Debt indicators, the context and implications of the figures, and provide visualizations for the reader.
"""
display(HTML("<h3 style='text-align:center;'>Introduction<br><br></h3>\
              <p{0}>{1}</p>".format(paragraph_style,intro_text)))

In [9]:
method_text = \
"""
Data was retrieved from The Federal Reserve through this third-party API: {} and the Congressional Budget Office. All
visualizations were created with the Bokeh library with Python 3. Though the Federal
Reserve was created in 1913, the federal debt was not measured
until after the onset of the stock market crash of 1929.
""".format("<a target='_blank' href='https://github.com/mortada/fredapi'>https://github.com/mortada/fredapi</a>")
display(HTML("<p style='text-align:center;font-size:20px;'>Methods<br></p>\
              <p style='text-align:justify;font-size:16px;'>" + method_text + "</p>"))

In [10]:
def hover(hover_color="#ffff99"):
            return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
                
styles = [
        hover(),
        dict(selector=".col0", props=[("text-align", "left"),]),
        dict(selector=".col1", props=[("text-align", "center"),]),
        dict(selector="caption", props=[("caption-side", "bottom")]),
        dict(selector="table",props=[("margin-left","auto"),
                                    ("margin-right","auto"),
                                    ("background-color","white")])
]

In [11]:
source = pd.DataFrame({'New Hampshire':[300000],
'Massachusetts':[4000000],
'Rhode Island and Providence Plantations':[200000],
'Connecticut':[1600000],
'New York':[1200000],
'New Jersey':[800000],
'Pennsylvania':[2200000],
'Delaware':[200000],
'Maryland':[800000],
'Virginia':[3500000],
'North Carolina':[2400000],
'South Carolina':[4000000],
'Georgia':[300000]}).T.reset_index()
source.columns = ['State','Debt']
source = source.sort_values(by='Debt', ascending=False)
source = source.append({'State':'Total','Debt':source['Debt'].sum()}, ignore_index=True)
source['4/9 Certificate Interest'] = round(source['Debt'] * (4/9) * 0.06,2)
source['3/9 Certificate Interest'] = round(source['Debt'] * (3/9) * 0.03,2)
source['2/9 Certificate Interest'] = round(source['Debt'] * (2/9) * 0.06,2)

In [12]:
HTML("""
<h3 style='text-align:center;'>Origin of the National Debt</h3>
<p{0}>
    During the American Revolution, a cash-strapped Continental Congress accepted loans from France. 
    Paying off these and other debts incurred during the Revolution proved one of the major challenges of the post-independence period. 
    The new U.S. Government attempted to pay off these debts in a timely manner, but the debts were at times a source of diplomatic tension.
</p>
<p{0}>
    In order to pay for its significant expenditures during the Revolution, 
    Congress had two options: print more money or obtain loans to meet the budget deficit. 
    In practice it did both, but relied more on the printing of money, which led to hyperinflation. 
    At that time, Congress lacked the authority to levy taxes, 
    and to do so would have risked alienating an American public that had gone to war with the British over the issue of unjust taxation.
</p>
<p{0}>
    During the Revolution, the French Government also provided the Americans with loans, 
    eventually totaling over two million dollars, most of which were negotiated by Benjamin Franklin. 
    John Adams also secured a loan from Dutch bankers in 1782.
    All in, the USA owed about \$11.7 million to foreigners and about $42 million to domestic creditors.
</p>
<p{0}>
    With the formation of the new government in 1789 and under the recently-adopted US Constitution, 
    the settlement of the Revolutionary War debt was a matter of prime importance. 
    As a result, the first House of Representatives directed the first Secretary of the Treasury, 
    Alexander Hamilton, during the presidential administration of George Washington, 
    to draw up a plan for the support of public credit. 
    Consequently, the First Report on the Public Credit was issued on January 9, 1790, 
    which became the foundation for subsequent action taken by Congress for funding and paying the public debt.
    The Funding Act of 1790 that followed was concerned primarily with funding the domestic debt held by the states.<sup><a href='https://founders.archives.gov/documents/Hamilton/01-06-02-0076-0002-0001'>[1]</a></sup> 
    </p>
<h3 style='text-align:center;'>The Funding Act of 1790</h3>
<p{0}>
    The Funding Act authorized the federal government to receive certificates of state war-incurred debts and to issue federal securities in exchange.
    By the Act, Congress authorized the assumption of a total of $21.5 million of state debts as shown below:<br /><br />
    <div style='display: flex;align-self:center;justify-content:center;'>""".format(paragraph_style) + \
    source.style.set_table_styles(styles)\
                        .set_caption("Hover to highlight.")\
                        .hide_index()\
                        .format({"Debt":"${:,.0f}",
                                 '2/9 Certificate Interest':"${:,.2f}",
                                 '3/9 Certificate Interest':"${:,.2f}",
                                 '4/9 Certificate Interest':"${:,.2f}"})\
                        .render()
             +"""</div>
    </p>
<p{0}>
    The purpose of these funds was to pay off the Federal Government's creditors and allow the states to use the interest
    gained to invest in infrastructure and industry.<sup><a href='https://history.state.gov/milestones/1784-1800/loans#:~:text=During%20the%20American%20Revolution%2C%20a,of%20the%20post%2Dindependence%20period.'>[2]</a></sup>
    This sum was loaned to the Federal Government with the terms such that each subscriber was to be entitled to a certificate
    equivalent of four ninths of the sum subscribed, bearing 6% annual interest, 
    another certificate equal to three ninths of the sum subscribed bearing 3% annual interest with both commencing January 1, 1792, 
    and a third certificate of the remaining two ninths of the sum bearing 6% interest starting from the year 1800.
</p>
<p{0}>
    The Federal Government no longer owed money to foreign governments by 1795 and approximately one-fifth of state revenues came from Federal Government interest payments for the next few decades.

</p>
<p{0}>
    The establishment of a viable national credit allowed the United States of America to obtain loans at much lower rates on the international market,
    leading to the Louisiana Purchase of 1803. Throughout the formative years of the Federal Government, Treasury Secretary Alexander Hamilton's 
    tenacity in reducing the national debt followed by Thomas Jefferson resulted in lowering the public debt-to-GDP ratio from 30% to 10%.
    </p>
""".format(paragraph_style))

State,Debt,4/9 Certificate Interest,3/9 Certificate Interest,2/9 Certificate Interest
Massachusetts,"$4,000,000","$106,666.67","$40,000.00","$53,333.33"
South Carolina,"$4,000,000","$106,666.67","$40,000.00","$53,333.33"
Virginia,"$3,500,000","$93,333.33","$35,000.00","$46,666.67"
North Carolina,"$2,400,000","$64,000.00","$24,000.00","$32,000.00"
Pennsylvania,"$2,200,000","$58,666.67","$22,000.00","$29,333.33"
Connecticut,"$1,600,000","$42,666.67","$16,000.00","$21,333.33"
New York,"$1,200,000","$32,000.00","$12,000.00","$16,000.00"
New Jersey,"$800,000","$21,333.33","$8,000.00","$10,666.67"
Maryland,"$800,000","$21,333.33","$8,000.00","$10,666.67"
New Hampshire,"$300,000","$8,000.00","$3,000.00","$4,000.00"


In [134]:
display(HTML("""
<h3 style='text-align:center;'>A Closer Look at the Public Debt</h3>
<p{0}>
    The definition of public debt is how much a country owes to lenders outside of itself. These can include individuals, businesses, and even other governments.
    This figure is not the entirety of the federal debt. The other part is intragovernmental debt: debt that one part of the government owes to another part (e.g. Social Security Trust Funds).
</p>
<p{0}>
    There are pros and cons to a government taking on debt. The main advantage is raising funds without increasing taxes in the short run.
    Eventually, if too much debt is taken on, the government's interest payments become too great and the risk they won't be able to pay back their creditors increases,
    thereby increase the interest rate demanded. The tradeoff there is the government has to spend more money on interest payments instead of public services.
    The ratio of publicly held debt to GDP is a better measure of a country's fiscal situation than just the nominal debt figure because it shows the burden of debt to the public
    relative to the country's total economic output and therefore its ability to finance or repay it. 
    Below, I show the debt as a % of GDP and highlight important historical periods of increasing debt. 
</p>
<p{0}>    
    The Congressional Budget Office provides projections for the next 10 years.
    <sup><a href='https://www.cbo.gov/publication/56165#:~:text=At%20%2416.8%20trillion%2C%20debt%20held,percent%20of%20GDP%2C%20by%202030.'>[4]</a></sup>
    <sup><a href="https://www.cbo.gov/publication/56073#:~:text=Federal%20debt%20held%20by%20the%20public%20is%20projected%20to%20rise,recorded%20in%20the%20United%20States.">[5]</a></sup>
    </p>
""".format(paragraph_style)))

In [86]:
from bokeh.models import BoxAnnotation, Square, Range1d, Tabs, Panel, Label, Arrow, OpenHead, Legend, LegendItem
from bokeh.palettes import Spectral9


df = pd.read_csv(r"C:/Users/ryana/Downloads/federal-debt-held-by-the.csv")
df.index = pd.to_datetime(df['Category'], format='%Y')
df['Debt Held by the Public'] = df['Debt Held by the Public'] / 100
df.drop("Category",axis=1, inplace=True)

hover = HoverTool(
    names = ['a',],
    tooltips = [
        ('year','@Category{%Y}'),
        ('public debt','@{Debt Held by the Public}{0.%f}')
        
    ],
    formatters = {'@Category':'datetime'},
    mode='vline',
    line_policy="nearest"

)

a = figure(sizing_mode = 'scale_width',
             plot_height=250,
             tools=[hover,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Percent",
             title="Public Debt as a % of GDP")
a.line(x='Category',y='Debt Held by the Public',name='a',source=df,color='darkorchid')
a.title.align='center'
a.yaxis.formatter=NumeralTickFormatter(format="0%")
a.y_range = Range1d(0,1.21)

period_legend = Legend()

for period_name,\
    left_date,\
    left_date_format,\
    right_date,\
    right_date_format,\
    color in zip(["Civil War","World War I","Great Depression","New Deal Era","World War II","Savings & Loan Crisis","Great Recession","US Credit Downgrade","CBO Projection"],
                 ["April 12, 1861","July 28, 1914","August 1929","1934","September 1, 1939","1980","December 2007","August 2011","2020"],
                 ["%B %d, %Y"     ,"%B %d, %Y"   ,"%B %Y"      ,"%Y"  ,"%B %d, %Y"        ,"%Y"  ,"%B %Y"        ,"%B %Y"      ,"%Y"],
                 ["April 9, 1865","November 11, 1918",'March 1933','1938',"September 2, 1945",'1994','June 2009','January 2012',"2030"],
                 ["%B %d, %Y"    ,"%B %d, %Y"        ,"%B %Y"     ,"%Y"  ,"%B %d, %Y"        ,"%Y"  ,"%B %Y"    ,"%B %Y"       ,"%Y"],
                 Spectral9):
    period_box = BoxAnnotation(left=pd.to_datetime(left_date,format=left_date_format), 
                               right=pd.to_datetime(right_date,format=right_date_format),
                               fill_alpha=0.5,
                               fill_color=color)
    a.add_layout(period_box)
    circ = a.circle(0,0,color=color,size=0)
    legend_item = LegendItem(label=period_name, renderers=[circ])
    period_legend.items.append(legend_item)
    
    
# names = ["Civil War","World War I","Great Depression","New Deal Era","World War II","Savings & Loan Crisis","Great Recession","US Credit Downgrade","CBO Projection"]
# items = []
# for i in range(len(Spectral9)):
#     items += [(names[i],[a.circle(i,i,color=Spectral9[i],size=0)])]
# a.add_layout(Legend(items=items),'left')
a.add_layout(period_legend,"left")

In [129]:
l = list(range(2020,2031))
l.insert(0,2020)

debt_proj = pd.DataFrame({"Year":l,
                          "GDP":[21219.85,20649.245,20997.435,22076.92,22975.3625,23955.69,25010.4775,26129.785,27287.0675,28461.51,29591.3575,30732.2825],
                          "Debt":[22853.77845,23790,24901,26112,27295,28476,29790,31109,32252,33583,34805,36204]})
debt_proj['Debt-to-GDP'] = debt_proj['Debt'] / debt_proj['GDP']
debt_proj['Year'] = pd.to_datetime(debt_proj['Year'], format="%Y")

In [ ]:
source = pd.DataFrame(federal_debt_to_gdp/100, columns=['federal debt'])

hover = HoverTool(
    names=['fd',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('federal debt'   ,'@{federal debt}{0.%f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
    ,line_policy="nearest"
)
hover_proj = HoverTool(
    names=['proj',],
    tooltips=[
        ('date'        ,'@Year{%Y}'),
        ('federal debt'   ,'@{Debt-to-GDP}{0.%f}')
    ],
    formatters={
        '@Year':'datetime',
        
    }
    ,mode='vline'
    ,line_policy="nearest"
)

fd = figure(sizing_mode = 'scale_width',
             plot_height=250,
             tools=[hover,hover_proj,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Percent",
             title="Federal Debt as a % of GDP")

fd.line(x='index',y='federal debt',name='fd',source=source,color='darkorchid')
fd.line(x='Year',y='Debt-to-GDP',name='proj',source=debt_proj,color='darkorchid')
fd.title.align='center'
fd.yaxis.formatter=NumeralTickFormatter(format="0%")
fd.y_range = Range1d(0,1.21)

period_legend = Legend()

for period_name,\
    left_date,\
    left_date_format,\
    right_date,\
    right_date_format,\
    color in zip(["Savings & Loan Crisis","Great Recession","US Credit Downgrade","CBO Projection"],
                 ["1980"                 ,"December 2007"  ,"August 2011"        ,"2020"],
                 ["%Y"                   ,"%B %Y"          ,"%B %Y"              ,"%Y"],
                 ['1994'                 ,'June 2009'      ,'January 2012'       ,"2030"],
                 ["%Y"                   ,"%B %Y"          ,"%B %Y"              ,"%Y"],
                 Spectral9[-4:]):
    period_box = BoxAnnotation(left=pd.to_datetime(left_date,format=left_date_format), 
                               right=pd.to_datetime(right_date,format=right_date_format),
                               fill_alpha=0.5,
                               fill_color=color)
    fd.add_layout(period_box)
    circ = fd.circle(0,0,color=color,size=0)
    legend_item = LegendItem(label=period_name, renderers=[circ])
    period_legend.items.append(legend_item)
    
fd.add_layout(period_legend,"left")

tab1 = Panel(child=a, title="Public Debt")
tab2 = Panel(child=fd, title="Federal Debt")

tabs = Tabs(tabs=[tab1,tab2])
show(tabs)

In [136]:
display(HTML("""
<h3 style='text-align:center;'>Implications of Federal Debt Increase Periods</h3>
"""+\
# <h4 style='text-align:center;'>The Civil War</h4>
# <p{0}>
#     <a href="https://fee.org/articles/the-economic-costs-of-the-civil-war/">https://fee.org/articles/the-economic-costs-of-the-civil-war/</a>
#     </p>
# <h4 style='text-align:center;'>World War I</h4>
# <h4 style='text-align:center;'>The New Deal</h4>
# <p{0}>
#     In the 1930's, you can see there is a a significant increase in the debt level before World War II. This is due to President Franklin D. Roosevelt's New Deal
#     which increased the debt by 150% to over $40 billion. The New Deal was a series of programs, public work projects, financial reforms, and regulations. Here are a few of the outcomes
#     of the New Deal:

#     During World War II, the US didn't prioritize reducing the debt. 
    
#         <div style="display:table;margin: 0 auto;">
#             <ul>
#                 <li>https://www.investopedia.com/articles/investing/011116/economic-effects-new-deal.asp#the-bottom-line</li>
#             </ul>
#         </div>
#     </p>
# <h4 style='text-align:center;'>World War II</h4>
# <h4 style='text-align:center;'>The Reagan Era</h4>
# <p{0}>
#     https://www.investopedia.com/articles/financial-theory/banking-crisis-1980s.asp
#     </p>
# <h4 style='text-align:center;'>The Great Recession</h4>
# <h4 style='text-align:center;'>The Sovereign Debt Crisis</h4>
# <h4 style='text-align:center;'>The Coronavirus Pandemic</h4>
"""
<p{0}>
    Obviously, during times of war, there is a higher propensity for the government to borrow money. 
    The government pays conscripted soldiers and invests in private companies manufacturing 
    materials for the war. 
</p>
<p{0}>
    With increasing federal debt, interest payments gradually increase and the cost to borrow more increases.
    The government eventually runs into the problem where tax revenues they receive must either be
    allocated to debt or government services. In 2011, the US government already ran into that problem.
    
</p>
""".format(paragraph_style)))

In [ ]:
source = pd.DataFrame(federal_debt_total * 10 ** 3, columns=['debt'])

hover = HoverTool(
    names=['tfd',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('federal debt'   ,'@{debt}{$0,0} million')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)

tfd = figure(sizing_mode = 'scale_width',
             plot_height=250,
             tools=[hover,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Millions of Dollars",
             title="Total Federal Debt")
tfd.line(x='index',y='debt',name='tfd',source=source,color='orangered')
tfd.title.align='center'
tfd.yaxis.formatter=NumeralTickFormatter(format="$0,0")

In [16]:
df = pd.merge(pd.DataFrame(federal_debt_total,columns=['debt']),pd.DataFrame(population,columns=['pop']),left_index=True,right_index=True,how='inner')
source1 = pd.DataFrame(df['debt'] *10**6 / df['pop'], columns = ['debt per capita'])

hover = HoverTool(
    names=['fdp',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('debt per capita'   ,'@{debt per capita}{$0,0}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)

tfdc = figure(sizing_mode = 'scale_width',
             plot_height=250,
             tools=[hover,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Dollars",
             title="Total Federal Debt Per Capita")
tfdc.line(x='index',y='debt per capita',name='fdp',source=source1,color='orangered')
tfdc.line(x=(nominal_gdp/population).index,y=(nominal_gdp/population),color='green')
tfdc.title.align='center'
tfdc.yaxis.formatter=NumeralTickFormatter(format="$0,0")

tab1 = Panel(child=tfd, title="Total Federal Debt")
tab2 = Panel(child=tfdc, title="Federal Debt per Capita")


tabs = Tabs(tabs=[tab1,tab2])
show(tabs)

In [ ]:
federal_deficit.index = federal_deficit.index-timedelta(days=1)
source = pd.DataFrame((federal_deficit.groupby(pd.Grouper(freq='Q')).last()), columns=['federal deficit'])

hover = HoverTool(
    names = ['fde',],
    tooltips = [
        ('date','@index{%F}'),
        ('federal deficit','@{federal deficit}{$0,0f} million')
        
    ],
    formatters = {'@index':'datetime'},
    mode='vline',
    line_policy="nearest"

)
fde = figure(sizing_mode = 'scale_width',
             plot_height=250,
             tools=[hover],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Millions of Dollars",
             title="Federal Surplus/Deficit")
fde.line(x='index',y='federal deficit',name='fde',source=source,color='saddlebrown')
fde.title.align='center'
fde.yaxis.formatter=NumeralTickFormatter(format="$0,0")

In [15]:
source1 = pd.DataFrame(annual_federal_deficit/100, columns=['annual deficit'])

hover = HoverTool(
    names = ['fde1',],
    tooltips = [
        ('date','@index{%F}'),
        ('federal deficit','@{annual deficit}{%0.0f}')
        
    ],
    formatters = {'@index':'datetime'},
    mode='vline',
    line_policy="nearest"

)
fdea = figure(sizing_mode = 'scale_width',
             plot_height=250,
             tools=[hover],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Percent",
             title="Federal Surplus/Deficit as a % of GDP")
fdea.line(x='index',y='annual deficit',name='fde1',source=source1,color='saddlebrown')
fdea.title.align='center'
fdea.yaxis.formatter=NumeralTickFormatter(format="%0.0")

tab1 = Panel(child=fdea, title="Federal Surplus/Deficit as a % of GDP")
tab2 = Panel(child=fde, title="Federal Surplus/Deficit")


tabs = Tabs(tabs=[tab1,tab2])
show(tabs)

In [17]:
df = pd.DataFrame({"Federal Banks":debt_by_fed_banks/federal_debt_total,
                   "Public":debt_by_public/federal_debt_total,
                   "Foreign Investors":debt_by_foreigners/federal_debt_total,
                   "Private Investors":debt_by_private_investors/federal_debt_total,
                  "Agencies & Trusts":debt_by_agencies/federal_debt_total})
hover = HoverTool(
    names=['debt_components',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('publicly held'   ,'@{Public}{0.0%}'),
        ('held by private investors'   ,'@{Private Investors}{0.0%}'),
        ('held by foreign investors'   ,'@{Foreign Investors}{0.0%}'),
        ('held by federal banks'   ,'@{Federal Banks}{0.0%}'),
        ('held by agencies & trusts','@{Agencies & Trusts}{0.0%}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)

comp = figure(sizing_mode = 'scale_width',
             plot_height=250,
             tools=[hover,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Percent",
             y_range=(0, 1),
             title="Major Holders of Federal Debt")

comp.line(x='index',y='Federal Banks',source=df,color='orangered', legend_label='Federal Banks')
comp.line(x='index',y='Public',name='debt_components',source=df,color='black',legend_label='Public')
comp.line(x='index',y='Private Investors',source=df,color='green',legend_label='Private Investors')
comp.line(x='index',y='Foreign Investors',source=df,color='red',legend_label='Foreign Investors')
comp.line(x='index',y='Agencies & Trusts',source=df,color='blue',legend_label='Agencies & Trusts')
comp.legend.location=(0,100)
comp.legend.label_text_font_size = "8pt"
comp.title.align='center'
comp.yaxis.formatter=NumeralTickFormatter(format="0.0%")

show(comp)

In [18]:
def hover(hover_color="#ffff99"):
            return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
                
styles = [
        hover(),
        dict(selector="th", props=[("font-size", "100%"),
                                ("text-align", "center"),]),
        dict(selector="td", props=[("text-align", "center"),]),
        dict(selector="caption", props=[("caption-side", "bottom")]),
        dict(selector="table",props=[("margin-left","auto"),
                                    ("margin-right","auto"),
                                    ("background-color","white")])
]

In [19]:
HTML("""<div style='display: flex;align-self:center;justify-content:center;'><br>""" + \
df.corr(method="kendall").style.set_table_styles(styles)\
                    .set_caption("Hover to highlight.")\
                    .format("{:.2f}%")\
                    .render()
    + """</div>""")

,Federal Banks,Public,Foreign Investors,Private Investors,Agencies & Trusts
Federal Banks,1.00%,0.11%,-0.13%,-0.16%,-0.07%
Public,0.11%,1.00%,-0.43%,0.72%,-1.00%
Foreign Investors,-0.13%,-0.43%,1.00%,-0.32%,0.46%
Private Investors,-0.16%,0.72%,-0.32%,1.00%,-0.86%
Agencies & Trusts,-0.07%,-1.00%,0.46%,-0.86%,1.00%


In [20]:
# "https://www.theatlantic.com/business/archive/2012/11/the-long-story-of-us-debt-from-1790-to-2011-in-1-little-chart/265185/#:~:text=The%20debt%2Dto%2DGDP%20ratio,incurred%20during%20World%20War%20II."

'https://www.theatlantic.com/business/archive/2012/11/the-long-story-of-us-debt-from-1790-to-2011-in-1-little-chart/265185/#:~:text=The%20debt%2Dto%2DGDP%20ratio,incurred%20during%20World%20War%20II.'